<h1>URE</h1>

The unified rule engine (URE) is a generic opencog rule engine operating on the AtomSpace. Two chaining modes are currently supported, Forward Chaining and Backward Chaining.

In this tutorial, we are going to have a quick look at how we can do both forward and backward chaining.

In [ ]:
(use-modules
  (opencog)
  (opencog query)
  (opencog rule-engine))

Now let's create the knowledge base, characterising frogs and spiders.

In [ ]:
; Frog croaks
(Evaluation (stv 1.0 1.0)
   (Predicate "croaks") (Concept "frog"))

; Spider bites
(Evaluation (stv 1.0 1.0)
   (Predicate "bites") (Concept "spider"))

; Frog is green
(Inheritance (stv 1.0 1.0)
   (Concept "frog") (Concept "green"))

; Spider is brown
(Inheritance (stv 1.0 1.0)
   (Concept "spider") (Concept "brown"))

; Frog eats files
(Evaluation (stv 1.0 1.0)
   (Predicate "eats_flies") (Concept "frog"))

; Spider eats files
(Evaluation (stv 1.0 1.0)
   (Predicate "eats_flies") (Concept "spider"))

Next, let's create the inference rules. There are rules that are ready to use in URE, but to make it clearer for demostration purpose, let's simplified the process and create the our own rules for this tutorial:

In [ ]:
(define if-croaks-and-eats-flies-then-frog-rule
  (BindLink
    (Variable "$X")
    (And
      (Evaluation
        (Predicate "croaks")
        (Variable "$X"))
      (Evaluation
        (Predicate "eats_flies")
        (Variable "$X")))
    (Inheritance
      (Variable "$X")
      (Concept "frog")))
)

(define if-croaks-and-eats-flies-then-frog-rule-name (DefinedSchema "if-croaks-and-eats-flies-then-frog-rule"))
(Define if-croaks-and-eats-flies-then-frog-rule-name if-croaks-and-eats-flies-then-frog-rule)

(define if-frog-then-green-rule
  (Bind
    (Variable "$X")
    (Inheritance
      (Variable "$X")
      (Concept "frog"))
    (Inheritance
      (Variable "$X")
      (Concept "green")))
)

(define if-frog-then-green-rule-name (DefinedSchema "if-frog-then-green-rule"))
(Define if-frog-then-green-rule-name if-frog-then-green-rule)

And then create our own rule base and add those rules to it:

In [ ]:
(define frog-rb (Concept "frog-rb"))

(ure-add-rules frog-rb
  (list
    (cons if-croaks-and-eats-flies-then-frog-rule-name (stv 0.9 1))
    (cons if-frog-then-green-rule-name (stv 0.5 1))))

(ure-set-maximum-iterations frog-rb 20)

We also need to specifiy the source for forward chaining, to see what we can infer from it based on the knowledge we have.

In [ ]:
(define source
  (Evaluation (stv 1.0 1.0)
     (Predicate "croaks")
     (Concept "frog")))

Finally do forward chaining using `cog-fc`:

In [ ]:
(cog-fc frog-rb source)

We can do backward chaining in a similar manner, but this time, instead of starting from a source, we start from the target -- what we want to find out.

So let's define the target, see what is green in color:

In [ ]:
(define target (Inheritance (Variable "$what") (Concept "green")))

Additionally declare the variables involved:

In [ ]:
(define var_decl (TypedVariable (Variable "$what") (Type "ConceptNode")))

And finally, do backward chaining using `cog-bc`:

In [ ]:
(cog-bc frog-rb target #:vardecl var_decl)